# Pickup Pose Estimation

importing<br>
OpenCV: 3.4.1 <br>
NumPy: 1.13.3 <br>

In [ ]:
import io
import socket
import struct
import cv2
import numpy as np
import paramiko
import matplotlib.pyplot as plt
import cv2.aruco as aruco
import glob
import math
import websockets
import asyncio
import time
import copy
import os 
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import clear_output
from os import path
from PIL import Image

%matplotlib inline

# Coputer Vision Functions

### Calibration parameters

calibrated on 1920x1080  <br>
only use the same res.

In [ ]:
def loadCameraParameters():
    calibrationPath = "calibrationData"
    ret = np.load(path.join(calibrationPath,"ret.npy"))
    dist = np.load(path.join(calibrationPath,"dist.npy"))
    mtx = np.load(path.join(calibrationPath,"mtx.npy"))  
    rvecs = np.load(path.join(calibrationPath,"rvecs.npy"))  
    tvecs = np.load(path.join(calibrationPath,"tvecs.npy"))  
    return (ret, dist,mtx,rvecs, tvecs)

## Marker Functions
Define the marker properties <br>
make sure that the marker size (number of small squares in a marker) and its dimensions are correct

In [ ]:
def loadMarkers():
    markerList = []
    for i in range (10):
        arucoIdNum = i #np.random.randint(0,100)
        if arucoSize == 7:
            arucoDim = 700
            arocoDic = aruco.DICT_7X7_250
        else:
            arucoDim = int(500)
            arocoDic = aruco.DICT_6X6_250
        aruco_dict = aruco.Dictionary_get(arocoDic)
        marker = aruco.drawMarker(aruco_dict,arucoIdNum,arucoDim)
        markerList.append(marker)
    return (markerList, arocoDic, aruco_dict)

Detection and pose estimation

In [ ]:
def detectAruco(frame):
    # detects markers in a frame
    aruco_dict = aruco.Dictionary_get(arocoDic)
    parameters =  aruco.DetectorParameters_create()
    corners, ids, rejectedImgPoints = aruco.detectMarkers(frame, aruco_dict, parameters=parameters)
    frame = aruco.drawDetectedMarkers(frame, corners, ids, (0,255,0))
    #displayFrame(frame)
    return (corners, ids)

def markerPoseEstimation(frame):
    corners, ids = detectAruco(frame)
    if arucoSize == 7:
        rvec,tvec,c = aruco.estimatePoseSingleMarkers(corners,70,mtx, dist)
    else:
        rvec,tvec,c = aruco.estimatePoseSingleMarkers(corners,50,mtx, dist)

    try:
        frame = aruco.drawDetectedMarkers(frame, corners, ids, (0,255,0))
        cv2.imwrite("pickup.jpg", frame)
        for i in range(len(rvec)):
            frame = aruco.drawAxis(frame,mtx,dist,rvec[i],tvec[i],100)
        displayFrame(frame)
        
        return (rvec,tvec,frame,ids)
    except:
        return None
   

### Helper Functions 

In [ ]:
def displayFrame(frame):
    dpi = 500
    im_data = frame
    height, width, depth = im_data.shape
    # What size does the figure need to be in inches to fit the image?
    figsize = width / float(dpi), height / float(dpi)

    # Create a figure of the right size with one axes that takes up the full figure
    fig = plt.figure(figsize=figsize)
    ax = fig.add_axes([0, 0, 1, 1])

    # Hide spines, ticks, etc.
    ax.axis('off')
    # Display the image.
    ax.imshow(im_data, cmap='gray')
    #plt.imwrite(im_data, "this.jpg")
    plt.show()
    return frame

### Reading images from the Raspberry
This will open a pipe to Raspberry Pi and dowanlod an image <br>
make sure on the Raspberry side you have the pipe openede, with this machine's ip address

In [ ]:
def loadImages(label = ""):
    # Start a socket listening for connections on 0.0.0.0:8000 (0.0.0.0 means
    # all interfaces)
    
    print ("begins reading image")
    server_socket = socket.socket()
    server_socket.bind(('0.0.0.0', 8000))
    server_socket.listen(0)
    # Accept a single connection and make a file-like object out of it
    connection = server_socket.accept()[0].makefile('rb')

    counter = 1000
    i = 0
    
    # it must wait for 2 second to let the camera and connection start working
    time.sleep(2)
    tmpName = "./markerRead/tmp.jpg"

    try:
        while True:
            # first it makes sure that the file has not been downloaded yet
            # if so, it will delete it
            print ("checking for image")
            if os.path.isfile(tmpName):
                    s = os.stat(tmpName)
                    print ("tmp file exist")
                    if s.st_size > 10000:
                        print ("tmp file is large enough, it will be deleted!")
                        os.remove(tmpName)
                        break
            # Read the length of the image as a 32-bit unsigned int. If the
            # length is zero, quit the loop
            image_len = struct.unpack('<L', connection.read(struct.calcsize('<L')))[0]
            if not image_len:
                break
            # Construct a stream to hold the image data and read the image
            # data from the connection
            image_stream = io.BytesIO()
            image_stream.write(connection.read(image_len))
            # Rewind the stream, open it as an image with PIL and do some
            # processing on it
            image_stream.seek(0)
            image = Image.open(image_stream)
            name = "./markerRead/{}image{}.jpg".format(label,counter)
            image = np.array(image)
            cv2.imwrite(name, image)
            # it keeps the original image for further debugging and use a tmp version for 
            # checking in the next loop
            cv2.imwrite(tmpName, image)
            displayFrame(image)
            counter += 1    
            
    finally:
        try:
            # once it made sure that the image is downloaded, it deletes the tmp file
            os.remove(tmpName)
        except:
            print ("couldn't delete!")
        # close the pipe and says goodbye!
        connection.close()
        server_socket.close()
        


In [ ]:
def rotation_matrix(axis, degree):
    # from: https://stackoverflow.com/questions/6802577/rotation-of-3d-vector
    """
    Return the rotation matrix associated with counterclockwise rotation around
    a given axis by theta radians.
    """
    theta = math.radians(degree)
    axis = np.asarray(axis)
    axis = axis/math.sqrt(np.dot(axis, axis))
    a = math.cos(theta/2.0)
    b, c, d = -axis*math.sin(theta/2.0)
    aa, bb, cc, dd = a*a, b*b, c*c, d*d
    bc, ad, ac, ab, bd, cd = b*c, a*d, a*c, a*b, b*d, c*d
    return np.array([[aa+bb-cc-dd, 2*(bc+ad), 2*(bd-ac)],
                     [2*(bc-ad), aa+cc-bb-dd, 2*(cd+ab)],
                     [2*(bd+ac), 2*(cd-ab), aa+dd-bb-cc]])

In [ ]:
def findPickUp(img, markerTags, debug = True):
    """ 
    It takes an image as a numpy array plus the Aruco marker tags that it should look for
    for the aruco markers they are set in this order: center, start, end
    center is the marker at the center of the board, and start and end are makers on the object
    usually start == end 
    for example markerTags = [0,9,9]
    """
    print (markerTags)
    
    # edit this to match workobject and world coordiantion
    # this translates the camera coordination to robot coordination system
    translate = np.array([[0,-1,0],[-1,0,0],[0,0,1]])
    
    # detects the markers pose in the image
    rvec,tvec,frame,ids = markerPoseEstimation(img)
    
    # place holders
    objectMarkersIds = []
    objectMarkersLocs = []
    objectMarkersRots = []
    centerMarkerId = []
    centerMarkerLoc = []
    centerMarkerRot = []

    # checks to see which markers are paired (or, which objects have both start and end markers in the image)
    for i in range(len(ids)):
        index = int(ids[i])
        if index == markerTags[1] or index == markerTags[2]:
            objectMarkersIds.append(i)
            objectMarkersLocs.append(tvec[i])
            objectMarkersRots.append(rvec[i])
        if index == markerTags[0]:
            centerMarkerId = i
            centerMarkerLoc.append(tvec[i])
            centerMarkerRot.append(rvec[i])
    
    # store markers locations
    centerMarkerLoc = np.array(centerMarkerLoc).reshape(1,3)
    objectMarkersLocs = np.array(objectMarkersLocs).reshape(2,3)
    
    # finds the point between two markers to set pick up point
    pickUpLoc = np.average(objectMarkersLocs,axis = 0)
    
    # using the ccenter marker location, it finds the relative location of the other markers
    relativeLoc = objectMarkersLocs - centerMarkerLoc
    relativePickUp = pickUpLoc - centerMarkerLoc
    
    # translates locations to robot coordination system
    relativeLoc = np.dot(relativeLoc,translate)
    relativePickUp = np.dot(relativePickUp,translate)
    
    
    
    if debug:
        print ("centerMarkerLoc:")
        print (centerMarkerLoc)    
        print ("objectMarkersLocs:")
        print (objectMarkersLocs)
        print ("pickUpLoc:")
        print (pickUpLoc)
        print ("relativeLoc:")
        print (relativeLoc)
        print ("relativeLoc shape",relativeLoc.shape)
        print ("relativePickUp:")
        print (relativePickUp)
    
    # calculates axial rotation for each pickup
    centerMarkerRot = np.array(centerMarkerRot).reshape(1,3)
    centerMarkerRot = zRot(centerMarkerRot)
    objectMarkersRots = np.array(objectMarkersRots).reshape(2,3)

    # finds each marker's rotation in degrees
    r0 = zRot(objectMarkersRots[0])
    r1 = zRot(objectMarkersRots[1])
    
    # averages the rotations and compensates for the robot tool rotation 
    pickUpRot = 180-(((r0+r1)/2))

    if debug:
        print ("real rotation!!: ",pickUpRot)
        print ("center rotation: ", centerMarkerRot)
    
    relativePickUpRot = pickUpRot - centerMarkerRot
    
    # generates the vector for Machina-friendly rotation
    vec = np.array([[1,0],[0,-1]])
    realRotation = (rotatearoundZ(pickUpRot, vec))

    machinaData = [relativePickUp[0,0],relativePickUp[0,1],0,
                   realRotation[0,0],realRotation[0,1],0,
                   realRotation[1,0],realRotation[1,1],0]
    
    
    ##################################################
    # plotting the results
    ##################################################
    
    plt.figure(figsize=(1, .75), dpi=300)
    plotRange = 500
    plt.xlim([-plotRange,plotRange])
    plt.ylim([-plotRange,plotRange])

    
    plt.axis('equal')
    plt.grid(color='black', linestyle='-', linewidth=.1,)
    colors = ['r','b','y','g','c']
    size = 5
    alphaLevel = .5
    
    # plotting object markers
    for i in range (len(relativeLoc)):
        try:
            x = relativeLoc[i,0]
            y = relativeLoc[i,1]
            plt.scatter(x,y, c='blue', alpha=alphaLevel, s = size)
        except:
            pass
        
    # plotting the pcikup marker
    x = relativePickUp[0,0]
    y = relativePickUp[0,1]
    plt.scatter(x,y, c='yellow', alpha=alphaLevel, s = size)

    x = 0
    y = 0
    plt.scatter(x,y, c='red', alpha=alphaLevel, s = size)
    plt.show()
    
    
    return (relativePickUp,relativePickUpRot, machinaData)


def zRot(rot):
    # finds the degree of rotation around z axis from an rotation around a vector
    r = cv2.Rodrigues(rot)
    rotations = rotationMatrixToEulerAngles(r[0])
    data = math.degrees(rotations[2])
    return (data)

def rotationMatrixToEulerAngles(R) :
    # just in case we may need it later, rotationMatrix to Euler
    sy = math.sqrt(R[0,0] * R[0,0] +  R[1,0] * R[1,0])
     
    singular = sy < 1e-6
 
    if  not singular :
        x = math.atan2(R[2,1] , R[2,2])
        y = math.atan2(-R[2,0], sy)
        z = math.atan2(R[1,0], R[0,0])
    else :
        x = math.atan2(-R[1,2], R[1,1])
        y = math.atan2(-R[2,0], sy)
        z = 0
 
    return np.array([x, y, z])

def rotatearoundZ(deg, pair):
    # rotates two planar vectors on XY plane around the z vector
    r = math.radians(deg)
    rotM = np.array([[math.cos(r),math.sin(r)],[-math.sin(r),math.cos(r)]])
    # sample of pair: np.array([[1,0],[0,-1]])
    # the first one is x and the second one is y
    rotated = np.dot(pair,rotM)
    newRot = np.zeros((2,1))
    return (np.concatenate((rotated, np.zeros((2,1))), axis = 1))

def rotMatrix(r):
    r = math.radians(r)
    print (math.cos(r),math.sin(r))
    return np.array([[math.cos(r),math.sin(r)],[-math.sin(r),math.cos(r)]])

# Execution Helper Functions

In [ ]:
def runCommands(cmds):
    # parses a list of commands one by one to the Bridge
    for cmd in cmds:
        try:
            asyncio.get_event_loop().run_until_complete(sendToBridge(machinaAddress,cmd ))
        except:
            print ("FAIL: {}".format(cmd))
    return 

In [ ]:
async def sendToBridge(path,command):
    # Send a message as string to a given web socket
    # it always waits for a response!
    print("sending to bridge...")

    async with websockets.connect(path) as websocket:
        # it waits until the ws sends the message
        await websocket.send(command)
        print("Message sent:{}".format(command))
        # then waits Bridge sends the feedback
        feedback = await websocket.recv()
        # then it prints the feedback
        print("Message Recieved:{}".format(feedback))

In [ ]:
def initialSetup(wobj,externalAxis): 
    # defines the tool, set precision and speed and move the robot to the correct track position and pose
    relativePose = scanPosition + wobj
    commands = ["SpeedTo({});".format(fastSpeed), 
                "PrecisionTo(0.0);",
                "Tool.create(\"gripper\",1.04087,-0.0813326,337.978,0.5,0.866025,0 ,-0.5, 0.866025,0, 10, 0, 0, 55);",
                "Attach(\"gripper\");",
                "ExternalAxesTo({},null,null,null,null,null);".format(externalAxis),
                "TransformTo({},{},{},1,0,0,0,-1,0);".format(relativePose[0],relativePose[1],relativePose[2])]
    return commands

In [ ]:
def findPickUpPose(label, markerTags):
    # reads the saved image to find the pickup pose
    name = "./markerRead/{}image1000.jpg".format(label)
    img = cv2.imread(name)
    loc,rot, machinaData = findPickUp(img,markerTags)
    return (loc,rot, machinaData)

# PcikupNPlace

## Setup

In [ ]:
# setting up the values for this project
arucoSize = 5
machinaAddress = "ws://127.0.0.1:6999/Bridge"


# this is a workaround for not having the workobjects
# we use them as workobjects, without the orientation
pickTable = np.array([7437.17,-1618.14,-428.35])
pickExternalAxis = 6300

sortTable = np.array([3780.63,-1997.29,-513.81])
sortExternalAxis = 5000
safeSortExternalAxis = 4000

placeTable = np.array([-171.89,-2102.00,-517.73])
placeExternalAxis = 1500
safePlaceExternalAxis = 1500

# x,y value is based on the camera distance from the TCP
# z value is set to cover more area to be scanned
scanPosition = np.array([-225.70,0,1400])

# loading camera calibration data
ret, dist,mtx,rvecs, tvecs = loadCameraParameters()
markerList, arocoDic, aruco_dict = loadMarkers()


# basic parameters
fastSpeed = 500
slowSpeed = 150

safeDist = 200
waitTime = 1500

# this number defines how close the gripper would get to the object
# this is different from the TCP since the tool has different active depths
mainZVal = 90
pickUpZVal = mainZVal
sortZVal = mainZVal
placeZVal = mainZVal


#############################
## change it to 90 for final
## for test, set it to 200
#############################
# a simple switch to set if the robot is going to grab objects or stays in a safe distance
dryRun = False

if dryRun:
    pickUpZVal = 200
    sortZVal = 200
    placeZVal = 200

# these come from Dynamo (or the other "platform")
placingLocations = np.array([[1069.2, 1069.2, 90, 0],
                            [2288.4, 459.6, 90, 90],
                            [1323.2, 866, 90, 90],
                            [1069.2, 662.8, 90, 0],
                            [2034.4, 459.6, 90, 90],
                            [612.0, 459.6, 90, 90],
                            [231.0, 459.6, 90, 0],
                            [1069.2, 256.4, 90, 0],
                            [1323.2, 53.2, 90,90],
                            [1069.2, -150, 90, 0],
                            [-150, 459.6, 90, 90]])
                       
sortLocations = np.array([[1230.15, 1046, 90],
                        [1820.7, 842.8, 90],
                        [1009.869957, 842.8, 90],
                        [1230.15, 639.6, 90],
                        [586.17, 842.0, 90],
                        [222.469957, 842.8, 140.8],
                        [2068.35, 436.4, 90],
                        [1230.15, 233.2, 90],
                        [1477.8, 436.4, 90],
                        [1230.15, 30, 90],
                        [639.6, 436.4, 90]])

if len(placingLocations) != len(sortLocations):
    raise ValueError('Number of placing location should match the number of placing location.')
    
centerMarkerID = 0 
partListSize = len(sortLocations)

# makes list of partIndice in the given format. for example: [0,6,6]
partIndexList = [[centerMarkerID,i,i] for i in range(partListSize+1)]

# just some overriding on the data that comes from Dynamo to match our Z values
# because we don't want to smash the table bro!
placingLocations = np.array([[v[0],v[1],placeZVal, v[3]] for v in placingLocations])
sortLocations =    np.array([[v[0],v[1],sortZVal] for v in sortLocations])

## Main workflow
### Scan
There are several switches, like "start" in this block, make sure they are set for your purpose

In [ ]:
"""
calculate the pose
marker tags are:[center marker id, 
first side of the element id, end side of the lement id]
"""

goToScanPose = "TransformTo(7354.26,-1653.22,1170.6,1,0,0,0,-1,0);"


####################################
# switch to True to run commands
###################################
real = True

def sortLoop():
    cmds =initialSetup(pickTable, pickExternalAxis)
    if real:
        runCommands(cmds)
        pass
    #
    while (True):

        pairTags = scanTable(pickTable)
        print (pairTags)
        sortPicknPlace(pairTags)
        break
    return
 
def placeLoop():
    # change it
    for i in range (partListSize):
        print ("Picking element #{}".format(i))
        cmds = pickFromSort(i)
        if real:
            
            runCommands(cmds)
            pass
        else:
            for cmd in cmds:
                print (cmd)
        print ("Placing element #{}".format(i))
        cmds = placeAtInstall(i)
        if real:
            runCommands(cmds)
        else:
            for cmd in cmds:
                print (cmd)
    return
     
def pickFromSort(index):
    loc = list(sortLocations[index])
    loc.extend([1,0,0,0,-1,0])
    cmds = approaching(loc,sortTable,False)
    return cmds  

def placeAtInstall(index):
    loc = list(placingLocations[index])
    loc.extend([1,0,0,0,-1,0]) 
    #placing(index, direction,wobj,sorting = False):
    direction = [-1,-1]
    if index == 1:
        direction = [1,-1]
    cmds = placing(index, direction,placeTable,False)
    return cmds

def scanTable(wobj):   
    # for each scan phase we will do this:

    # look at the table, finds what items or on the table
    img = cv2.imread("./markerRead/scan_image1000.jpg")
    rvec,tvec,frame,ids = markerPoseEstimation(img)
    flatIds = [item[0] for item in ids]
    print (flatIds)
    pairTags = [i  for i in range(partListSize+1) if flatIds.count(i) == 2]
    tagsIndex = dict()
    for tag in pairTags:
        index1 = flatIds.index(tag)
        index2 = flatIds.index(tag,index1+1)
        tagsIndex[tag] = (index1,index2)  
        
    print (pairTags)
    print(tagsIndex) 
    return (pairTags)
    


def sortPicknPlace(pairTags,debug = False):
    # plan to reach out to them one by one
    for tag in pairTags:
    
        print (partIndexList[tag])
    

        #locs,rots,machinaData = findPickUpPose("scan_",[0,1,1])
        locs,rots,machinaData = findPickUpPose("scan_",partIndexList[tag])
        #keep the safety distance
        machinaData[2] = pickUpZVal
        # grab object
        cmds = approaching(machinaData,pickTable)
        cmds
        if debug:
            print ("#######################################")
            print ("####### approaching to pcik up ########")
            print (cmds)
            print ("#######################################")
        if real:
            runCommands(cmds)
        
        cmds = placing(tag,[0,0],sortTable,True)
        if debug:
            print ("#######################################")
            print ("####### placing to sort table ########")
            print (cmds)
            print ("#######################################")
        if real:
            runCommands(cmds)
            

def approaching(data,wobj,sorting = True):
    # data is machinaData as a list 
    data = np.array(data)
    
    loc = data[:3]
    
    pickLoc = loc + wobj
    safePickLoc = pickLoc + (0,0,safeDist)
    
    safePick = copy.deepcopy(data)
    safePick[:3] = safePickLoc
    
    pick = copy.deepcopy(data)
    pick[:3] = pickLoc
    
    pickCmd = formatTranslateTo(pick)
    safePcikCmd = formatTranslateTo(safePick)
    relativePose = scanPosition + wobj
    
    if sorting:
        externalAxVal = pickExternalAxis
    
    else:
        if pickLoc[2] < 200:
            externalAxVal = safeSortExternalAxis 
            
        else:   
            externalAxVal = sortExternalAxis
            
    commands = ["SpeedTo({})".format(fastSpeed),
                "ExternalAxesTo({},null,null,null,null,null);".format(externalAxVal),
                "TransformTo({},{},{},1,0,0,0,-1,0);".format(relativePose[0],relativePose[1],relativePose[2]),
                safePcikCmd,
                "SpeedTo({})".format(slowSpeed),
                "WriteDigital(\"Do_2\",True,False);",
                "WriteDigital(\"Do_3\",False,False);",
                "Wait({});".format(waitTime),
                pickCmd,
                "Wait({});".format(waitTime),
                "WriteDigital(\"Do_3\",True,False);",
                "WriteDigital(\"Do_2\",False,False);",
                "Wait({});".format(waitTime),
                safePcikCmd,
                "SpeedTo({})".format(fastSpeed),
               ]
    if sorting: 
        commands.append(goToScanPose)
    return commands


def placing(index, direction,wobj,sorting = False):
    # data is machina data as a list, this is the target position that came 
    # direction is a np.array of shape 2, defines in which direction we should appraoch the placing
    # [1,0]:approach the taget from positive x and slide to 0 
    # [1,1]:approaching from positive x and y in 45 degree angle to 0
    
    
    # first, calculate the placing position
    # this is the installation location of all elements 
    
    if sorting :
        objectData = sortLocations[index-1]
    else:
        objectData = placingLocations[index]
    
    placingLoc = objectData[:3]
    relativePlacing = placingLoc + wobj

    vec = np.array([[1,0],[0,-1]])
    
    if sorting:
        realRotation = rotatearoundZ(0, vec)
    else:
        if objectData[3] == 90:
            rot = 90
        else:
            rot = 0
        realRotation = rotatearoundZ(rot, vec)

    # this is the machina data for placing point
    data = np.array([relativePlacing[0],relativePlacing[1],relativePlacing[2],
                    realRotation[0,0],realRotation[0,1],0,
                    realRotation[1,0],realRotation[1,1],0])
    
    
    # find the safe point to move to
    safetyPointData = copy.deepcopy(data)
    
    safeFactor = np.array([direction[0]*safeDist,direction[1]*safeDist,safeDist])
    
    safetyPoint = safeFactor+relativePlacing

    safetyPointData[:3] = safetyPoint
    safeCmd = formatTranslateTo(safetyPointData)
    
    # find a crawling point
    crawlingPointData = copy.deepcopy(data)
    print (crawlingPointData)
    crawlingPoint = safetyPoint + np.array([0,0,safeDist])
    crawlingPointData[:3] = crawlingPoint
    
    
    if crawlingPointData[0] < -350 :
        
        crawlingPointData[0] = -350

    crawlCmd = formatTranslateTo(crawlingPointData)

    
    # placing location
    placeCmd = formatTranslateTo(data)
    
    # retract afterwards
    departinPointData = copy.deepcopy(data)
    departinPoint = relativePlacing + np.array([0,0,safeDist])
    departinPointData[:3] = departinPoint
    departCmd = formatTranslateTo(departinPointData)
    
    # setting the external axis value
    externalAxVal = relativePlacing[0]
    if externalAxVal < 0 : 
        externalAxVal = 0
    if relativePlacing[1] < -300:
        externalAxVal += 1000
        
    commands = ["ExternalAxesTo({},null,null,null,null,null);".format(externalAxVal),
                "SPeedTo({});".format (fastSpeed),
                #safeCmd, 
                "SPeedTo({});".format (slowSpeed),
                crawlCmd,
                placeCmd,
                "Wait({});".format(waitTime),
                "WriteDigital(\"Do_2\",True,False);",
                "WriteDigital(\"Do_3\",False,True);",
                "Wait({});".format(waitTime),
                "SPeedTo({});".format (fastSpeed),
                departCmd]
    return commands
    

def formatTranslateTo(messageRaw):
    # messageRaw as a list of numbers
    # formats a translation motion into Machina format
    data = ""
    for d in messageRaw:
        data = data+ str(d)+"," 
    data = data[:-1]
    cmd = "TransformTo({});".format(data)
    return cmd

def initAction(wobj, externalAxis):
    # workobject as np.array of shape 3
    # external axis as an int
    # generates the initial setup commands
    cmd = initialSetup(wobj, externalAxis)
    # sends commands for Bridge
    runCommands(cmd)
    print ("In scanning position")
    return 

### Sorting

In [ ]:
def sortingAction():
    initAction(pickTable,pickExternalAxis)
    loadImages("scan_")
    sortLoop()

# safety switch to start the sorting process
startSorting = False   
if startSorting :
    sortingAction()

## Installing

In [ ]:
def installingAction():
    # defines the tool, set precision and speed and move the robot to the correct track position and pose
    cmd = initialSetup(sortTable, sortExternalAxis)
    runCommands(cmd)
    print ("done")
    placeLoop() 
    
# safety switch to start the installation
startInstalling = True
if startInstalling:
    installingAction()